<a href="https://colab.research.google.com/github/aadityane93/SortingLine-ml/blob/main/Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing from Google Drive

In [3]:
!pip install tsfresh
# getml --upgrade
from google.colab import files
import numpy as np
import pandas as pd
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table
from sklearn.model_selection import train_test_split, KFold, cross_val_predict, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score
import xgboost as xgb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.5/176.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 56.6 MB/s eta 0:00:00


In [5]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
folder_id = '1ps_Bh93KQRJ4WbnilORmKd1BaWUXtmt1' #100 files

folder_list = drive.ListFile({'q': "'{}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false".format(folder_id)}).GetList()


for folder in folder_list:
    folder_title = folder['title']
    print(f"Processing folder: {folder_title}")
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder['id'])}).GetList()

    for index, file in enumerate(file_list):
        file_title = file['title']
        print(f"Downloading {file_title}")
        file.GetContentFile(file_title)






# hundred_hz_folder.GetContentFile('100hz-test-feature-50percent-100files.csv')
# hundred_hz_folder.GetContentFile('')

# #400hz csv files
# hundred_hz_folder = drive.CreateFile({'id':'1elpsYNvdZT3m0g8ACxi7Cm2h0PhhF6Wf'})
# hundred_hz_folder.GetContentFile('400hz-test-feature-50percent-10files.csv')
# hundred_hz_folder.GetContentFile('400hz-test-feature-50percent-100files.csv')
# hundred_hz_folder.GetContentFile('400hz-test-feature-20percent-1000files.csv')

# #800hz csv files
# hundred_hz_folder = drive.CreateFile({'id':'1Txfn_K6WRC_fTM8bDBkUfVUSbWFuKJHJ'})
# hundred_hz_folder.GetContentFile('800hz-test-feature-50percent-10files.csv')
# hundred_hz_folder.GetContentFile('800hz-test-feature-50percent-100files.csv')
# hundred_hz_folder.GetContentFile('800hz-test-feature-20percent-1000files.csv')

Processing folder: 400hz
Processing folder: 100hz
Processing folder: 800hz


# Reading CSV files

In [80]:
def read_file(filename):
  return pd.read_csv(filename, sep=',')

train_feature = pd.DataFrame(read_file('100hz-train-feature-50percent-100files.csv'))
test_feature = pd.DataFrame(read_file('100hz-test-feature-50percent-100files.csv'))
train_target = pd.DataFrame(read_file('100hz-train-target-50percent-100files.csv'))
test_target = pd.DataFrame(read_file('100hz-test-target-50percent-100files.csv'))

In [81]:
train_feature = train_feature.drop(columns=['Unnamed: 0', 'Datum', 'P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)','color'])
test_feature = test_feature.drop(columns=['Unnamed: 0', 'Datum', 'P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)','color'])
train_target = train_target.drop(columns=['Unnamed: 0'])['color']
test_target = test_target.drop(columns=['Unnamed: 0'])['color']

In [82]:
test_target

,color
0,0
1,0
2,0
3,0
4,0
...,...
195,3
196,3
197,3
198,3


In [83]:
train_feature

,Uhrzeit,I_In,V_In,I_Out(A),V_Out(V),Temp(°C),fileno
0,2024-05-02 11:37:37.940396:940,0.25,11.65,0.29,8.97,37.63,0
1,2024-05-02 11:37:37.948287:948,0.27,11.52,0.25,9.00,37.63,0
2,2024-05-02 11:37:37.956565:956,0.31,11.65,0.27,9.00,37.63,0
3,2024-05-02 11:37:37.968670:968,0.31,11.65,0.25,9.03,37.63,0
4,2024-05-02 11:37:37.977001:976,0.37,11.65,0.33,9.03,37.63,0
...,...,...,...,...,...,...,...
149815,2024-05-02 13:05:39.262050:262,0.27,11.61,0.25,9.00,38.13,349
149816,2024-05-02 13:05:39.270394:270,0.31,11.61,0.25,9.03,38.13,349
149817,2024-05-02 13:05:39.282562:282,0.31,11.61,0.31,9.00,38.38,349
149818,2024-05-02 13:05:39.290867:290,0.25,11.70,0.31,9.00,38.13,349


In [84]:
extracted_features =  extract_features(
    train_feature,
    column_id="fileno",
    column_sort="Uhrzeit",
    impute_function=impute
)

Feature Extraction: 100%|██████████| 1000/1000 [15:20<00:00,  1.09it/s]


In [85]:
# feature_file = f'features-{index*4}.csv'
# features.to_csv(feature_file, sep=',')
# files.download(feature_file)

In [86]:
extracted_features.index.values

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49, 100, 101,
       102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
       115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
       128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
       141, 142, 143, 144, 145, 146, 147, 148, 149, 200, 201, 202, 203,
       204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,
       217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229,
       230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242,
       243, 244, 245, 246, 247, 248, 249, 300, 301, 302, 303, 304, 305,
       306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318,
       319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 33

In [89]:
print(extracted_features.shape)
print(train_target.shape)

(200, 3915)
(200,)


In [90]:
train_target

,color
0,0
1,0
2,0
3,0
4,0
...,...
195,3
196,3
197,3
198,3


In [91]:
extracted_features = extracted_features.reset_index(drop=True)
extracted_features

,I_In__variance_larger_than_standard_deviation,I_In__has_duplicate_max,I_In__has_duplicate_min,I_In__has_duplicate,I_In__sum_values,I_In__abs_energy,I_In__mean_abs_change,I_In__mean_change,I_In__mean_second_derivative_central,I_In__median,...,Temp(°C)__fourier_entropy__bins_5,Temp(°C)__fourier_entropy__bins_10,Temp(°C)__fourier_entropy__bins_100,Temp(°C)__permutation_entropy__dimension_3__tau_1,Temp(°C)__permutation_entropy__dimension_4__tau_1,Temp(°C)__permutation_entropy__dimension_5__tau_1,Temp(°C)__permutation_entropy__dimension_6__tau_1,Temp(°C)__permutation_entropy__dimension_7__tau_1,Temp(°C)__query_similarity_count__query_None__threshold_0.0,Temp(°C)__mean_n_absolute_max__number_of_maxima_7
0,0.0,0.0,1.0,1.0,189.99,60.2121,0.042299,0.000305,-8.051530e-06,0.29,...,1.204209,1.795131,3.805391,1.318114,2.152424,2.987462,3.746941,4.391745,0.0,37.88
1,0.0,0.0,1.0,1.0,185.58,59.6738,0.042040,0.000216,0.000000e+00,0.29,...,1.436031,2.089244,4.045644,1.410305,2.314589,3.235639,4.075079,4.799483,0.0,37.63
2,0.0,0.0,1.0,1.0,184.09,59.0125,0.043344,0.000250,-4.625929e-20,0.30,...,1.139872,1.786103,3.792321,1.462869,2.450789,3.414714,4.310813,5.059538,0.0,37.63
3,0.0,0.0,0.0,1.0,187.40,60.1416,0.042311,0.000246,-1.642036e-05,0.29,...,1.219551,1.839760,3.856714,1.485377,2.524549,3.559655,4.512030,5.219138,0.0,37.63
4,0.0,0.0,1.0,1.0,183.78,59.4834,0.041930,0.000285,1.680672e-05,0.29,...,1.211701,1.898553,3.854095,1.467557,2.479623,3.458075,4.381721,5.129161,0.0,37.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.0,0.0,0.0,1.0,265.83,84.9157,0.044959,0.000046,3.480278e-05,0.31,...,1.341356,2.030510,4.034222,1.121341,1.823993,2.502287,3.153191,3.726377,0.0,38.38
196,0.0,0.0,1.0,1.0,268.86,85.3792,0.045148,-0.000023,-1.140251e-05,0.31,...,1.347392,2.003717,3.983037,1.186467,1.918449,2.630214,3.280199,3.873044,0.0,38.38
197,0.0,0.0,1.0,1.0,272.37,86.9875,0.045383,0.000000,5.636979e-05,0.31,...,1.439519,2.077125,4.031914,1.125912,1.819074,2.512666,3.146670,3.703434,0.0,38.38
198,0.0,0.0,1.0,1.0,263.17,83.9259,0.047226,0.000070,-2.333722e-05,0.31,...,1.333746,1.993594,3.862643,1.126697,1.857773,2.575251,3.223993,3.818383,0.0,38.38


In [92]:
selected_features = select_features(extracted_features, train_target)

In [93]:
selected_features.head()

,I_Out(A)__sum_of_reoccurring_data_points,"Temp(°C)__fft_coefficient__attr_""abs""__coeff_0",Temp(°C)__sum_values,"Temp(°C)__fft_coefficient__attr_""real""__coeff_0","I_In__fft_aggregated__aggtype_""variance""",I_In__sum_of_reoccurring_data_points,Temp(°C)__abs_energy,Temp(°C)__sum_of_reoccurring_data_points,"I_Out(A)__fft_coefficient__attr_""angle""__coeff_1",I_Out(A)__range_count__max_1__min_-1,...,"I_Out(A)__fft_coefficient__attr_""abs""__coeff_52","I_In__fft_coefficient__attr_""abs""__coeff_18","I_In__fft_coefficient__attr_""abs""__coeff_49","Temp(°C)__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.4","I_Out(A)__fft_coefficient__attr_""abs""__coeff_75","Temp(°C)__change_quantiles__f_agg_""var""__isabs_False__qh_0.4__ql_0.2","Temp(°C)__change_quantiles__f_agg_""mean""__isabs_True__qh_0.4__ql_0.2","Temp(°C)__change_quantiles__f_agg_""var""__isabs_True__qh_0.4__ql_0.2",I_Out(A)__partial_autocorrelation__lag_4,"I_Out(A)__linear_trend__attr_""pvalue"""
0,189.20,23397.99,23397.99,23397.99,10023.276497,186.58,878768.6987,23397.99,12.281726,622.0,...,1.936640,2.818966,0.864501,0.009772,1.365618,0.026331,0.105332,0.015238,0.313336,9.732668e-12
1,183.39,22658.52,22658.52,22658.52,8893.805173,182.01,850026.7576,22658.52,13.918459,602.0,...,1.709872,2.444789,2.194649,0.115036,1.655305,0.000000,0.000000,0.000000,0.190863,2.882170e-12
2,182.81,22567.51,22567.51,22567.51,9239.818040,181.48,846013.1913,22567.51,13.999959,601.0,...,1.248486,2.373812,0.837620,0.122736,1.623350,0.000000,0.000000,0.000000,0.228127,1.075370e-11
3,183.34,22888.18,22888.18,22888.18,9143.044817,183.44,857410.4134,22888.18,14.953357,610.0,...,1.589452,2.286361,1.707862,0.120370,1.456777,0.000000,0.000000,0.000000,0.264175,1.424840e-12
4,182.58,22371.61,22371.61,22371.61,8209.689272,178.62,838352.7793,22371.61,11.591896,596.0,...,1.036741,2.623803,2.911375,0.122650,1.204775,0.000000,0.000000,0.000000,0.233434,1.172800e-09


In [97]:
x_train,x_test,y_train,y_test = train_test_split(selected_features,train_target, test_size=0.5, random_state=42, stratify=train_target)

In [98]:
model = RandomForestClassifier()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.97
              precision    recall  f1-score   support

           0       0.96      1.00      0.98        25
           1       0.96      0.96      0.96        25
           2       1.00      0.96      0.98        25
           3       0.96      0.96      0.96        25

    accuracy                           0.97       100
   macro avg       0.97      0.97      0.97       100
weighted avg       0.97      0.97      0.97       100

[[25  0  0  0]
 [ 0 24  0  1]
 [ 1  0 24  0]
 [ 0  1  0 24]]


In [99]:
def xgb_(X, y, num_folds=5):

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=24)

    f1_scores = []
    accuracies = []
    precisions = []

    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X)):
        print(f"Fold {fold_idx + 1}:")

        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Initialize and train the XGBoost model with regularization
        xgb1 = xgb.XGBClassifier(
            objective="multi:softmax",
            random_state=42,
            alpha=0.1,
            reg_lambda=0.1,
            max_depth=4,
            min_child_weight=10,
            subsample=0.8,
            colsample_bytree=0.8
        )

        # Fit the model
        xgb1.fit(X_train.to_numpy(), y_train.to_numpy().ravel())

        # Make predictions
        y_pred = xgb1.predict(X_test.to_numpy())

        # Ensure y_test is 1D for comparison
        y_test = y_test.to_numpy().ravel().astype(int)
        y_pred = y_pred.astype(int)

        # Calculate and store F1-score for this fold
        f1 = f1_score(y_test, y_pred, average='weighted')
        f1_scores.append(f1)

        # Calculate and store accuracy and precision for this fold
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        precision = precision_score(y_test, y_pred, average='weighted')
        precisions.append(precision)

        print(classification_report(y_test, y_pred))
        print(f"Fold {fold_idx + 1} F1-score: {f1:.4f}")
        print(f"Fold {fold_idx + 1} Accuracy: {accuracy:.4f}")
        print(f"Fold {fold_idx + 1} Precision: {precision:.4f}")

        # Calculate and print confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        print("Confusion Matrix:")
        print(cm)

        print("-" * 30)

    # Calculate and print average metrics across all folds
    avg_f1 = np.mean(f1_scores)
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)

    print(f"Average F1-score across {num_folds} folds: {avg_f1:.4f}")
    print(f"Average Accuracy across {num_folds} folds: {avg_accuracy:.4f}")
    print(f"Average Precision across {num_folds} folds: {avg_precision:.4f}")

    # Final model: Train on the entire dataset
    print("Training final model on the entire dataset...")
    X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(X, y, test_size=0.2, random_state=42)

    # Final model with regularization
    final_model = xgb.XGBClassifier(
        objective="multi:softmax",
        random_state=42,
        alpha=0.1,
        reg_lambda=0.1,
        max_depth=5,
        min_child_weight=10,
        subsample=0.8,
        colsample_bytree=0.8
    )

    # Train on the entire dataset
    final_model.fit(X_train_final.to_numpy(), y_train_final.to_numpy().ravel())

    # Evaluate the final model on the test set
    y_pred_final = final_model.predict(X_test_final.to_numpy())

    # Calculate accuracy, precision, confusion matrix for final model
    final_accuracy = accuracy_score(y_test_final, y_pred_final)
    final_precision = precision_score(y_test_final, y_pred_final, average='weighted')
    final_cm = confusion_matrix(y_test_final, y_pred_final)

    print(f"Final Model Accuracy: {final_accuracy:.4f}")
    print(f"Final Model Precision: {final_precision:.4f}")
    print("Final Model Confusion Matrix:")
    print(final_cm)

    # Save the final model
    # with open('XGB_final_model.pkl', 'wb') as f:
    #     pickle.dump(final_model, f)
    # files.download('XGB_final_model.pkl')



In [100]:
xgb_ (selected_features, y_train_df)

Fold 1:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00         9
           3       1.00      1.00      1.00        13

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40

Fold 1 F1-score: 1.0000
Fold 1 Accuracy: 1.0000
Fold 1 Precision: 1.0000
Confusion Matrix:
[[ 8  0  0  0]
 [ 0 10  0  0]
 [ 0  0  9  0]
 [ 0  0  0 13]]
------------------------------
Fold 2:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        10
           3       1.00      1.00      1.00        11

    accuracy                           1.00        40
   macro avg       1.00      1.00      1

In [ ]:
features_test = extract_features(
    x_test_df,
    column_id="fileno",
    column_sort="Uhrzeit",
    impute_function=impute
)

Feature Extraction:  24%|██▍       | 245/1000 [02:44<09:13,  1.37it/s]

In [ ]:
#Using the same columns from the training data
features_test = features_test[selected_features.columns]

In [ ]:
print(features_test.shape)
y_test_df= df_color_test['color']
y_test_df = y_test_df.reset_index(drop=True)
print(y_test_df.shape)

(320, 795)
(320,)


In [ ]:
# model = RandomForestClassifier()
# model.fit(features_test, y_test)
y_pred = model.predict(features_test)
accuracy = accuracy_score(y_test_df, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test_df, y_pred))
print(confusion_matrix(y_test_df, y_pred))

Accuracy: 0.86875
              precision    recall  f1-score   support

           0       0.99      0.99      0.99        80
           1       0.90      0.56      0.69        80
           2       0.71      0.99      0.82        80
           3       0.96      0.94      0.95        80

    accuracy                           0.87       320
   macro avg       0.89      0.87      0.86       320
weighted avg       0.89      0.87      0.86       320

[[79  0  1  0]
 [ 0 45 32  3]
 [ 1  0 79  0]
 [ 0  5  0 75]]


In [ ]:
# xgb_ (features_test, y_test_df)

In [ ]:
# Read file into DataFrame

wio15_df = pd.DataFrame(pd.read_csv(('wiO-15'), sep=',', header=0))



wio15_df['fileno'] = 15
wio15_df = wio15_df.drop(columns=['Datum','P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)'])

extracted_features = extract_features(
    wio15_df,
    column_id="fileno",
    column_sort="Uhrzeit",
    impute_function=impute
)
wio_features = extracted_features[selected_features.columns]
y_pred = model.predict(wio_features)
print(y_pred)
accuracy = accuracy_score(pd.Series([0]), y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(pd.Series([0]), y_pred))
print(confusion_matrix(pd.Series([0]), y_pred))


Feature Extraction: 100%|██████████| 5/5 [00:03<00:00,  1.46it/s]


[0]
Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

[[1]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [ ]:
# #For getML
# x_train_df2 = df_train.drop(columns=['P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)'])
# x_test_df2 = df_test.drop(columns=['P_In(W)','P_Out(W)','Energie_In(Wh)','Energie_Out(Wh)'])

# # x_train_df2 = x_train_df.copy(deep=True)

# import getml

# getml.engine.launch()
# getml.set_project("sorting-line")

# # Split at the last colon
# x_train_df2["Uhrzeit"] = (
#     x_train_df2["Uhrzeit"]
#     .str.rsplit(pat=':', n=1)
#     .str[0]
# )

# # Convert to datetime with flexible parsing
# x_train_df2["Uhrzeit"] = pd.to_datetime(
#     x_train_df2["Uhrzeit"],
#     format='mixed',
#     errors='coerce'
# )
# pop = getml.data.DataFrame.from_pandas(x_train_df2, name = "pop")


# pop.set_role("fileno", getml.data.roles.join_key)
# pop.set_role(["I_In", "V_In", "I_Out(A)", "V_Out(V)", "Temp(°C)"], getml.data.roles.numerical)
# pop.set_role("Uhrzeit", getml.data.roles.time_stamp)
# pop.set_role("color", getml.data.roles.target)

In [ ]:
# x_train_df2

In [ ]:
# pop